# 추천 시스템 - 중요⭐

Data가 없는 경우  
-> 고객의 행동에서 특정 콘텐츠의 특징을 추출하여 다음 추천을 결정(콘텐츠 기반 필터링) - 정확도는 떨어짐     
  
Data가 쌓인 경우  
-> 고객들간의 지난 행동이 유사한 것이라 가정하여 다른 사람의 패턴과 유사함을 비교하여 추천(협업 필터링)

In [77]:
import pandas as pd

raw_df = pd.read_csv('../data/tmdb_5000_movies.csv')
raw_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""nam...",http://www.avata...,19995,"[{""id"": 1463, ""n...",en,Avatar,In the 22nd cent...,150.437577,"[{""name"": ""Ingen...","[{""iso_3166_1"": ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""...",Released,Enter the World ...,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""nam...",http://disney.go...,285,"[{""id"": 270, ""na...",en,Pirates of the C...,Captain Barbossa...,139.082615,"[{""name"": ""Walt ...","[{""iso_3166_1"": ...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""...",Released,At the end of th...,Pirates of the C...,6.9,4500
2,245000000,"[{""id"": 28, ""nam...",http://www.sonyp...,206647,"[{""id"": 470, ""na...",en,Spectre,A cryptic messag...,107.376788,"[{""name"": ""Colum...","[{""iso_3166_1"": ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""...",Released,A Plan No One Es...,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""nam...",http://www.theda...,49026,"[{""id"": 849, ""na...",en,The Dark Knight ...,Following the de...,112.312950,"[{""name"": ""Legen...","[{""iso_3166_1"": ...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""...",Released,The Legend Ends,The Dark Knight ...,7.6,9106
4,260000000,"[{""id"": 28, ""nam...",http://movies.di...,49529,"[{""id"": 818, ""na...",en,John Carter,John Carter is a...,43.926995,"[{""name"": ""Walt ...","[{""iso_3166_1"": ...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""...",Released,Lost in our worl...,John Carter,6.1,2124


In [78]:
df = raw_df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'popularity', 'overview']]
pd.set_option('max_colwidth', 20)
df

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,"[{""id"": 28, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""n...",150.437577,In the 22nd cent...
1,285,Pirates of the C...,"[{""id"": 12, ""nam...",6.9,4500,139.082615,"[{""id"": 270, ""na...",139.082615,Captain Barbossa...
2,206647,Spectre,"[{""id"": 28, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""na...",107.376788,A cryptic messag...
3,49026,The Dark Knight ...,"[{""id"": 28, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""na...",112.312950,Following the de...
4,49529,John Carter,"[{""id"": 28, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""na...",43.926995,John Carter is a...
...,...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""id"": 28, ""nam...",6.6,238,14.269792,"[{""id"": 5616, ""n...",14.269792,El Mariachi just...
4799,72766,Newlyweds,"[{""id"": 35, ""nam...",5.9,5,0.642552,[],0.642552,A newlywed coupl...
4800,231617,"Signed, Sealed, ...","[{""id"": 35, ""nam...",7.0,6,1.444476,"[{""id"": 248, ""na...",1.444476,"""Signed, Sealed,..."
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],0.857008,When ambitious N...


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   popularity    4803 non-null   float64
 8   overview      4800 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 337.8+ KB


In [80]:
from ast import literal_eval # 문자열을 자료형으로 변환(str -> list)

# 문자열 -> list, dict로 변환
#result = literal_eval(df.iloc[0]['genres']) # 각각 적용하여 결과 추출

df['genres'] = df['genres'].apply(literal_eval) # 함수 자체를 해당 컬럼에 적용
df['genres']

C:\Users\user\AppData\Local\Temp\ipykernel_22136\907423600.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(literal_eval) # 함수 자체를 해당 컬럼에 적용


0       [{'id': 28, 'nam...
1       [{'id': 12, 'nam...
2       [{'id': 28, 'nam...
3       [{'id': 28, 'nam...
4       [{'id': 28, 'nam...
               ...         
4798    [{'id': 28, 'nam...
4799    [{'id': 35, 'nam...
4800    [{'id': 35, 'nam...
4801                     []
4802    [{'id': 99, 'nam...
Name: genres, Length: 4803, dtype: object

In [81]:
df['keywords'] = df['keywords'].apply(literal_eval)
df['keywords']

C:\Users\user\AppData\Local\Temp\ipykernel_22136\3086860884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['keywords'].apply(literal_eval)


0       [{'id': 1463, 'n...
1       [{'id': 270, 'na...
2       [{'id': 470, 'na...
3       [{'id': 849, 'na...
4       [{'id': 818, 'na...
               ...         
4798    [{'id': 5616, 'n...
4799                     []
4800    [{'id': 248, 'na...
4801                     []
4802    [{'id': 1523, 'n...
Name: keywords, Length: 4803, dtype: object

In [82]:
total_names = []

for id_names in df['genres']:
    
    names = []
    
    for id_name in id_names:
        names.append(id_name['name'])
        
    total_names.append(' '.join(names))   
    
df['genres'] = total_names

C:\Users\user\AppData\Local\Temp\ipykernel_22136\1897628688.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = total_names


In [83]:
total_keywords = []

for id_keywords in df['keywords']:
    
    keywords = []
    
    for id_keyword in id_keywords:
        keywords.append(id_keyword['name'])
        
    total_keywords.append(' '.join(keywords))   
    
df['keywords'] = total_keywords 

C:\Users\user\AppData\Local\Temp\ipykernel_22136\3119218250.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = total_keywords


In [84]:
# pandas select
df.loc[0, 'genres']
df.iloc[0]['genres']

'Action Adventure Fantasy Science Fiction'

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(min_df=1, ngram_range=(1,2)) # 최소 1번 이상 나오고, 2개씩 조합한 경우의 수를 벡터 행으로 활용

genres_vector = count.fit_transform(df['genres'])
keywords_vector = count.fit_transform(df['keywords'])

count.vocabulary_ # 컬럼의 순서(횟수가 아님)

{'culture': 9037,
 'clash': 7010,
 'future': 15032,
 'space': 34485,
 'war': 40021,
 'colony': 7433,
 'society': 34214,
 'travel': 38236,
 'futuristic': 15087,
 'romance': 31412,
 'alien': 1077,
 'tribe': 38408,
 'planet': 27732,
 'cgi': 6161,
 'marine': 22347,
 'soldier': 34275,
 'battle': 3232,
 'love': 21496,
 'affair': 548,
 'anti': 1760,
 'power': 28328,
 'relations': 30072,
 'mind': 23178,
 'and': 1426,
 'soul': 34431,
 '3d': 94,
 'culture clash': 9039,
 'clash future': 7012,
 'future space': 15075,
 'space war': 34525,
 'war space': 40145,
 'space colony': 34494,
 'colony society': 7442,
 'society space': 34240,
 'space travel': 34523,
 'travel futuristic': 38253,
 'futuristic romance': 15094,
 'romance space': 31433,
 'space alien': 34487,
 'alien tribe': 1148,
 'tribe alien': 38409,
 'alien planet': 1122,
 'planet cgi': 27737,
 'cgi marine': 6165,
 'marine soldier': 22373,
 'soldier battle': 34281,
 'battle love': 3254,
 'love affair': 21498,
 'affair anti': 550,
 'anti war': 

In [86]:
genres_vector.shape # 전체 영화수, 장르의 조합수(벡터의 열수)

(4803, 276)

In [87]:
keywords_vector.shape # 전체 영화수, 키워드의 조합수(벡터의 열수)

(4803, 41554)

In [88]:
genres_vector_array = genres_vector.toarray() # 각 행이 각 영화
genres_vector_array

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4803, 276))

In [89]:
keywords_vector_array = keywords_vector.toarray()
keywords_vector_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4803, 41554))

In [90]:
from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도로 유사한 행 확인

genre_sim = cosine_similarity(genres_vector_array, genres_vector_array)
genre_sim[0].argsort()[::-1] # 기본값이 오름차순이라 역순으로 바꾼 후에 나오는 index 활용

array([   0,   46, 3494, ..., 3600, 3569, 4799], shape=(4803,))

In [91]:
keyword_sim = cosine_similarity(keywords_vector_array, keywords_vector_array)

In [92]:
df.iloc[46]

id                           127585
title           X-Men: Days of F...
genres          Action Adventure...
vote_average                    7.5
vote_count                     6032
popularity               118.078691
keywords        1970s mutant tim...
popularity               118.078691
overview        The ultimate X-M...
Name: 46, dtype: object

In [93]:
df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,Action Adventure...,7.2,11800,150.437577,culture clash fu...,150.437577,In the 22nd cent...
1,285,Pirates of the C...,Adventure Fantas...,6.9,4500,139.082615,ocean drug abuse...,139.082615,Captain Barbossa...
2,206647,Spectre,Action Adventure...,6.3,4466,107.376788,spy based on nov...,107.376788,A cryptic messag...
3,49026,The Dark Knight ...,Action Crime Dra...,7.6,9106,112.312950,dc comics crime ...,112.312950,Following the de...
4,49529,John Carter,Action Adventure...,6.1,2124,43.926995,based on novel m...,43.926995,John Carter is a...


## 추천 함수

In [94]:
# 컨텐츠 기반 추출방식은 지양하는 것이 요즘 추세
def recommend_movies_by_genre(df, genre_sorted_index, title_name, top_n=10):
    
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_index = sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)
    
    return df.iloc[similar_index]
    
genre_sim_sorted = genre_sim.argsort()[:,::-1]

recommend_movies(df, genre_sim_sorted, 'Superman', 5)



,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,Action Adventure...,7.2,11800,150.437577,culture clash fu...,150.437577,In the 22nd cent...
46,127585,X-Men: Days of F...,Action Adventure...,7.5,6032,118.078691,1970s mutant tim...,118.078691,The ultimate X-M...
3494,27549,Beastmaster 2: T...,Action Adventure...,4.6,17,1.478505,based on novel t...,1.478505,Mark Singer retu...
870,8536,Superman II,Action Adventure...,6.5,629,30.515175,saving the world...,30.515175,Three escaped cr...
14,49521,Man of Steel,Action Adventure...,6.5,6359,99.398009,saving the world...,99.398009,A young boy lear...


In [95]:
def recommend_movies_by_keyword(df, keyword_sorted_index, title_name, top_n=10):
    
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_index = sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)
    
    return df.iloc[similar_index]
    
keyword_sim_sorted = keyword_sim.argsort()[:,::-1]

recommend_movies(df, keyword_sim_sorted, 'Superman', 5)

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
813,1924,Superman,Action Adventure...,6.9,1022,48.507081,saving the world...,48.507081,Mild-mannered Cl...
10,1452,Superman Returns,Adventure Fantas...,5.4,1400,57.925623,saving the world...,57.925623,Superman returns...
1296,9531,Superman III,Comedy Action Ad...,5.3,490,22.164202,saving the world...,22.164202,Aiming to defeat...
14,49521,Man of Steel,Action Adventure...,6.5,6359,99.398009,saving the world...,99.398009,A young boy lear...
870,8536,Superman II,Action Adventure...,6.5,629,30.515175,saving the world...,30.515175,Three escaped cr...
